In [1]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
import os
from tqdm import tqdm
import copy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class pMHCDataset(Dataset):
    def __init__(self, root, filename, aaindex, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.filename = filename
        self.aaindex = aaindex
        super(pMHCDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in processed_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass##不需要下载
    
    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, sample in tqdm(self.data.iterrows(), total=self.data.shape[0]):#tqdm可以显示运行进程
            # Get node features
            node_feats = self._get_node_features(sample["pep"],sample["hla_seq"],self.aaindex)
            edge_index = self._get_edge_index(sample["pep"],sample["hla_seq"])
            label = self._get_labels(sample["type"])
            # Create data object
            data = Data(x=node_feats, edge_index=edge_index, y=label, index=0) 
            torch.save(data, os.path.join(self.processed_dir, f'data_{index}.pt'))

    def _get_node_features(self, pep, HLA, aaindex):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_seq = pep + HLA
        all_node_feats = []
        for index, aa in enumerate(all_seq):
            node_feats = []
            ##aaindex
            node_feats.extend(aaindex[aa].to_list())
            anchar = [0,len(pep)]
            seq_onehot = [0,0]
            seq_onehot[sum([index >= i for i in anchar])-1] = 1
            node_feats.extend(seq_onehot)
            all_node_feats.append(node_feats)
        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats)
        
    
    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label)
    
    def _get_edge_index(self, pep, hla):
        ##生成边
        nodes = list(range(0,len(pep)+len(hla)))
        edge_index = [[],[]]  
        for i,_ in enumerate(pep):
            nodes_cp = copy.deepcopy(nodes)
            nodes_cp.remove(i)
            edge_index[0].extend([i]*(len(nodes)-1))
            edge_index[1].extend(nodes_cp)
        for i,_ in enumerate(hla):
            i = i + len(pep)
            nodes_cp = copy.deepcopy(nodes)
            nodes_cp.remove(i)
            edge_index[0].extend([i]*(len(nodes)-1))
            edge_index[1].extend(nodes_cp)  
        edge_index = torch.tensor(edge_index)
        return edge_index
    
    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt')) 
        return data
    
##model
from torch_geometric.nn import TransformerConv, TopKPooling, GraphNorm
from torch.nn import Linear, BatchNorm1d, ModuleList, LeakyReLU
from gtrick import random_feature
from gtrick.pyg import VirtualNode
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn import MemPooling
from torch.nn import LeakyReLU
class GNN(torch.nn.Module):
    def __init__(self, feature_size, model_params):
        super().__init__()
        
        embedding_size = model_params["model_embedding_size"]
        dense_neurons = model_params["model_dense_neurons"]
        n_heads = model_params["model_heads"]
        n_layers = model_params["model_layers"]
        self.n_layers = n_layers
        self.top_k_every_n = 1
        self.conv_layers = ModuleList([])
        self.transf_layers = ModuleList([])
        self.pooling_layers = ModuleList([])
        self.bn_layers = ModuleList([])
        self.vns = ModuleList()
        self.relu = LeakyReLU()

        # Transformation layer
        self.conv1 = TransformerConv(feature_size, 
                                    embedding_size, 
                                    heads=n_heads,
                                    beta=True) 

        self.transf1 = Linear(embedding_size*n_heads, embedding_size)
        self.bn1 =  GraphNorm(embedding_size)
        # Other layers
        for i in range(n_layers):
            self.conv_layers.append(TransformerConv(embedding_size, 
                                                    embedding_size, 
                                                    heads=n_heads,
                                                    beta=True))

            self.transf_layers.append(Linear(embedding_size*n_heads, embedding_size))
            self.bn_layers.append(GraphNorm(embedding_size))
            self.vns.append(VirtualNode(embedding_size, embedding_size))
            

        # Linear layers
        self.linear1 = Linear(embedding_size, dense_neurons)
        self.linear2 = Linear(dense_neurons, 1)  

    def forward(self, x, edge_index, batch_index):
        # Initial transformation
        x = self.conv1(x, edge_index)
        x = self.relu(self.transf1(x))
        x = self.bn1(x, batch_index)

        for i in range(self.n_layers):
            x, vx = self.vns[i].update_node_emb(x, edge_index, batch_index)
            x = self.conv_layers[i](x, edge_index)
            x = self.relu(self.transf_layers[i](x))
            x = self.bn_layers[i](x, batch_index)
            vx = self.vns[i].update_vn_emb(x, batch_index, vx)
        
        #x = x[node_index,:]
        # Output block
        x = self.relu(self.linear1(vx))
        x = self.linear2(x)
        return x

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train_one_epoch(epoch, model, train_loader, optimizer, loss_fn, epoch_num):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    all_preds_raw = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)): 
        #remove = batch[1]
        target = batch.y
        input_x = batch.x
        edge_index = batch.edge_index
        target = target.to(device)
        input_x = input_x.to(device)
        edge_index = edge_index.to(device)
        batch_index = batch.batch.to(device)
        node_index = batch.index.to(device)
        optimizer.zero_grad() 
        pred = model(input_x.float(), edge_index, batch_index) 
        # Calculating the loss and gradients
        loss = loss_fn(pred, target.reshape(pred.shape[0],1).float())
        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(target.cpu().detach().numpy())
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    all_preds_raw = np.concatenate(all_preds_raw).ravel()
    if epoch == epoch_num:
        return running_loss/step, all_preds, all_labels, all_preds_raw
    return running_loss/step, step, step, step

def test(epoch, model, test_loader, loss_fn):
    all_preds = []
    all_labels = []
    all_preds_raw = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        target = batch.y
        input_x = batch.x
        edge_index = batch.edge_index
        target = target.to(device)
        input_x = input_x.to(device)
        edge_index = edge_index.to(device)
        batch_index = batch.batch.to(device)
        node_index = batch.index.to(device)
        pred = model(input_x.float(), edge_index, batch_index) 
        loss = loss_fn(pred, target.reshape(pred.shape[0],1).float())
         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.rint(torch.sigmoid(pred).cpu().detach().numpy()))
        all_labels.append(target.cpu().detach().numpy())
        all_preds_raw.append(torch.sigmoid(pred).cpu().detach().numpy())
        
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    all_preds_raw = np.concatenate(all_preds_raw).ravel()
    return running_loss/step, all_preds, all_labels, all_preds_raw

In [3]:
def run_one_training(params, dataset, loss_fn, train_subsampler, test_subsampler):

    # Prepare training
    train_loader = DataLoader(dataset, batch_size=params["batch_size"], sampler=train_subsampler, num_workers = 18, pin_memory=True)
    test_loader = DataLoader(dataset, batch_size=params["batch_size"], sampler=test_subsampler, num_workers = 18, pin_memory=True)

    # Loading the model
    print("Loading model...")
    model_params = {k: v for k, v in params.items() if k.startswith("model_")}
    model = GNN(feature_size=22, model_params=model_params) 
    model = model.to(device)
    print(f"Number of parameters: {count_parameters(model)}")

    # < 1 increases precision, > 1 recall
    optimizer = torch.optim.Adam(model.parameters(), 
                                lr=params["learning_rate"],
                                weight_decay=params["weight_decay"])
    #scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params["scheduler_gamma"])

    # Start training
    for epoch in range(36): 
        # Training
        model.train()
        loss, all_preds, all_labels, all_preds_raw = train_one_epoch(epoch, model, train_loader, optimizer, loss_fn, epoch_num=35)
        print(f"Epoch {epoch} | Train Loss {loss}")
        # Testing
        if epoch == 35:
            model.eval()
            loss, all_preds_test, all_labels_test, all_preds_raw_test = test(epoch, model, test_loader, loss_fn)
            train_pred = pd.DataFrame({"all_preds":all_preds,"all_labels":all_labels,"all_preds_raw":all_preds_raw})
            test_pred = pd.DataFrame({"all_preds":all_preds_test,"all_labels":all_labels_test,"all_preds_raw":all_preds_raw_test})
    return train_pred, test_pred

In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils import data
from torch_geometric.loader import DataLoader
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm
HYPERPARAMETERS = {
    "batch_size": 8,
    "weight_decay": 0.00001,
    "learning_rate": 0.0001,
    "model_embedding_size": 64, 
    "model_dense_neurons": 32,
    "model_heads":3,
    "model_layers":3
}
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(1.4))

In [5]:
aaindex = pd.read_csv("../data/aaindex1_pca.csv")
train_dt = pMHCDataset(root="../data/models/train_data/",
                       filename="train_data_iedb_2.csv",
                       aaindex=aaindex)

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True)
for fold,(train_idx,test_idx) in enumerate(kfold.split(train_dt)):
    print(f'------------fold no---------{fold}----------------------')
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)
    train_pred, test_pred = run_one_training(HYPERPARAMETERS, train_dt, loss_fn, train_subsampler, test_subsampler)
    train_pred.to_csv("../data/models/gnn_train_res_"+str(fold)+".csv")
    test_pred.to_csv("../data/models/gnn_test_res_"+str(fold)+".csv")

------------fold no---------0----------------------
Loading model...
Number of parameters: 273089


100%|██████████| 947/947 [00:52<00:00, 18.03it/s]


Epoch 0 | Train Loss 0.7414756476123834


100%|██████████| 947/947 [00:51<00:00, 18.44it/s]


Epoch 1 | Train Loss 0.6813743410606444


...

100%|██████████| 947/947 [00:41<00:00, 22.71it/s]


Epoch 3 | Train Loss 0.638260764173117


100%|██████████| 947/947 [00:41<00:00, 22.70it/s]


Epoch 4 | Train Loss 0.6239594721177312


 15%|█▍        | 141/947 [00:08<00:33, 24.19it/s]